# 나이브 베이즈

In [2]:
import pandas as pd

df = pd.read_csv('sms_spam.csv', encoding='ISO-8859-1' )

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5559 entries, 0 to 5558
Data columns (total 2 columns):
type    5559 non-null object
text    5559 non-null object
dtypes: object(2)
memory usage: 86.9+ KB


In [4]:
df.head()

,type,text
0,ham,Hope you are having a good week. Just checking in
1,ham,K..give back my thanks.
2,ham,Am also doing in cbe only. But have to pay.
3,spam,"complimentary 4 STAR Ibiza Holiday or Â£10,000..."
4,spam,okmail: Dear Dave this is your final notice to...


In [5]:
df['text'][3]

'complimentary 4 STAR Ibiza Holiday or Â£10,000 cash needs your URGENT collection. 09066364349 NOW from Landline not to lose out! Box434SK38WP150PPM18+'

In [7]:
# text = df['text'][3]
# text = re.sub('\W', ' ', text)
# text = re.sub('\d', '', text)
# text = re.sub('[ \t]+' , ' ', text)
# text

In [11]:
import re
text = None
text_list = []
for i in range(len(df)):
    text = df['text'][i]
    text = re.sub('[^\w\s]', '', text)
    text = re.sub('\d', '', text)
    text = re.sub('[ \t]+' , ' ', text)
    text = text.lower()
    text_list.append(text)
#     text_list.append(re.split(' ', text))

In [12]:
print(text_list[:5])

['Hope you are having a good week Just checking in', 'Kgive back my thanks', 'Am also doing in cbe only But have to pay', 'complimentary STAR Ibiza Holiday or Â cash needs your URGENT collection NOW from Landline not to lose out BoxSKWPPPM', 'okmail Dear Dave this is your final notice to collect your Tenerife Holiday or CASH award Call from landline TCs SAE Box CWWX ppm']


In [13]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(min_df=0., max_df=1.0)
X = vect.fit_transform(text_list)
# print(pd.DataFrame(X.A, columns=vect.get_feature_names()).to_string()) #데이터 프레임 필요 없음.
# df_text = pd.DataFrame(X.toarray().transpose(), index = vect.get_feature_names()) #데이터 프레임 필요 없음

In [14]:
X

<5559x8589 sparse matrix of type '<class 'numpy.int64'>'
	with 70442 stored elements in Compressed Sparse Row format>

In [15]:
X.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [16]:
X.shape

(5559, 8589)

In [17]:
df.shape

(5559, 2)

In [18]:
import numpy as np
y = np.array(df.type.map({'ham' : 0, 'spam' : 1}))

In [19]:
y.shape

(5559,)

In [20]:
y

array([0, 0, 0, ..., 1, 1, 0], dtype=int64)

# Train, Test set 나누기

In [21]:
X.shape[0]*0.8

4447.2

In [22]:
X.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [23]:
from sklearn.model_selection import train_test_split

x_train, x_test = train_test_split(X, test_size=0.3, random_state=42)
x_train.shape, x_test.shape

((3891, 8589), (1668, 8589))

In [24]:
y_train, y_test = train_test_split(y, test_size=0.3, random_state=42)
y_train = np.array(y_train)
y_test = np.array(y_test)
y_train.shape, y_test.shape

((3891,), (1668,))

In [25]:
# x_train, x_test = np.array(np.split(X.toarray(), [round(X.shape[0]*0.8,)]))
# y_train, y_test = np.array(np.split(y, [round(y.shape[0]*0.8,)]))

In [26]:
# x_train.shape, x_test.shape

In [27]:
# y_train.shape, y_test.shape

# 1. 베르누이 분포 나이브 베이즈

In [28]:
from sklearn.naive_bayes import BernoulliNB
model_bern = BernoulliNB().fit(x_train, y_train)

In [29]:
pre = model_bern.predict(x_test)
pre

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [30]:
print((pre==y_test).mean())

0.9664268585131894


# scikit-learn의 교차검증 기능

### 단순 데이터 분리¶
- 단순 데이터를 나누는 것과 교차검증의 차이 확인.

In [31]:
dfX_train, dfX_test, dfy_train, dfy_test = train_test_split(X, y, test_size=0.3, random_state=42)
dfX_train.shape, dfy_train.shape, dfX_test.shape, dfy_test.shape

((3891, 8589), (3891,), (1668, 8589), (1668,))

- 교차 검증의 장점은 train_test_split와 비교해보면 train_test_split는 데이터를 무작위로 나눔. 데이터를 무작위로 나눌 때 
- 훈련 세트 ==>  분류하기 어려운 샘플
- 테스트세트 ==> 분류하기 쉬운 샘플
- 테스트 세트의 정확도는 비현실적으로 높게 나올 것입니다. 반대의 경우라면 테스트 세트의 정확도는 비현실적으로 낮게 나옴

- 그러나 교차 검증을 사용하면 테스트 세트에 각 샘플이 정확하게 한번씩 들어가고 각 샘플은 폴드 중 하나에 속하며 각 폴드는 한번씩 테스트 세트가 됨 그렇기 때문에 교차 검증의 점수를 높이기 위해서는 데이터 셋에 있는 모든 샘플에 대해 모델이 잘 일반화 되어야 함

- 데이터를 여러개로 나누면 모델이 훈련 데이터에 얼마나 민감한지 알 수가 있음 
- 교차 검증의 주요 단점은 연산 비용이 늘어남 모델을 k개 만들어야 하므로 데이터를 한 번 나눴을때보다 대략 k배 더 느림

# K-fold 교차검증

- scikit-learn 패키지의 model_selection 서브 패키지는 KFold 클래스를 비롯한 다양한 교차검증 생성기를 제공한다. 이 생성기의 split 메서드는 학습용과 검증용의 데이터 인덱스를 출력하는 파이썬 반복자(iterator)를 반환한다.

In [32]:
from sklearn.naive_bayes import BernoulliNB
model_K = BernoulliNB().fit(X, y)

In [33]:
%%time
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# cv = KFold(5, shuffle=True, random_state=42)
cross_val_score(model_K, X, y, scoring="accuracy", cv=5) ##분류이므로 scoring = accuracy, 회기는 r2

Wall time: 24.9 ms


In [34]:
cv = KFold(5, shuffle=True, random_state=42)
cross_val_score(model_K, X, y, scoring="accuracy", cv=cv) #교차검증에서 데이터 전체를 넣어야 됨.

array([0.96852518, 0.97032374, 0.97571942, 0.97571942, 0.97659766])

In [35]:
cross_val_score(model_K, X, y, scoring="accuracy", cv=5).mean(), cross_val_score(model_K, X, y, scoring="accuracy", cv=cv).mean()

(0.9726547317804558, 0.97337708591003)

# 2. 가우시안 정규 분포 나이브 베이즈

In [1]:
import pandas as pd
G_df = pd.read_csv('winemag-data_first150k.csv', index_col=0)
G_df.head()

,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude


In [2]:
list_col = ['description', 'points']
G_df = G_df[list_col].copy()
G_df.head()

,description,points
0,This tremendous 100% varietal wine hails from ...,96
1,"Ripe aromas of fig, blackberry and cassis are ...",96
2,Mac Watson honors the memory of a wine once ma...,96
3,"This spent 20 months in 30% new French oak, an...",96
4,"This is the top wine from La Bégude, named aft...",95


In [3]:
def rank_estimator(score):
    
    if (85 >= score >= 80):
        return "B"
    elif (90 >= score > 85):
        return "A"
    elif (95 >= score > 90):
        return "S"
    else:
        return "SS"

G_df.points = G_df.points.apply(rank_estimator)
G_df.head()

,description,points
0,This tremendous 100% varietal wine hails from ...,SS
1,"Ripe aromas of fig, blackberry and cassis are ...",SS
2,Mac Watson honors the memory of a wine once ma...,SS
3,"This spent 20 months in 30% new French oak, an...",SS
4,"This is the top wine from La Bégude, named aft...",S


In [4]:
import re
G_text = None
G_text_list = []
for i in range(len(G_df)):
    G_text = G_df['description'][i]
    G_text = re.sub('[^\w\s]', '', G_text)
    G_text = re.sub('\d', '', G_text)
    G_text = re.sub('[ \t]+' , ' ', G_text)
    G_text = G_text.lower()
    G_text_list.append(G_text)

In [5]:
G_text_list[0], G_text_list[100]

('this tremendous varietal wine hails from oakville and was aged over three years in oak juicy redcherry fruit and a compelling hint of caramel greet the palate framed by elegant fine tannins and a subtle minty tone in the background balanced and rewarding from start to finish it has years ahead of it to develop further nuance enjoy ',
 'juicy kiwi lime blossom and sour apple candy aromas show on the nose of this ripe whitethe first submitted from the region the palate is oily in texture with honeydew and more limekiwibubblegum flavors')

In [6]:
# vectorizer count
# from sklearn.feature_extraction.text import CountVectorizer
# vect = CountVectorizer(min_df=3)
# X = vect.fit_transform(G_text_list)

In [8]:
# vectorizer (tf-idf)

from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(stop_words='english')
X  = vect.fit_transform(G_text_list)
X

<150930x42667 sparse matrix of type '<class 'numpy.float64'>'
	with 3395875 stored elements in Compressed Sparse Row format>

In [9]:
X.shape

(150930, 42667)

In [10]:
from sklearn.model_selection import train_test_split
x_train, x_test = train_test_split(X, test_size=0.3, random_state=0)
x_train.shape, x_test.shape

((105651, 42667), (45279, 42667))

In [11]:
import numpy as np
y = G_df.points.values
y_train, y_test = train_test_split(y, test_size=0.3, random_state=42)
y_train = np.array(y_train)
y_test = np.array(y_test)
y_train.shape, y_test.shape

((105651,), (45279,))

In [12]:
x_train_array = x_train.toarray()

In [13]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB().fit(x_train_array, y_train)

MemoryError: 

In [ ]:
x_test_array = x_test.toarray()

In [ ]:
from sklearn.metrics import accuracy_score
pred = clf.predict(x_test_array)
accuracy_score(y_test, pred, )

In [ ]:
count = 0
for i in range(len(pred)):
    if pred[i] == np.array(y_test)[i]:
        count += 1

print((count/len(pred))*100, '%')

# 3. 다항 분포 나이브 베이즈 모형¶

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train.toarray(), y_train)

In [ ]:
pred = clf.predict(x_test)
accuracy_score(y_test, pred, )

# 4. 2,3 Cross Validation

In [14]:
# modeling
from sklearn.naive_bayes import MultinomialNB
Mult = MultinomialNB()

y = G_df.points.values
X

<150930x42667 sparse matrix of type '<class 'numpy.float64'>'
	with 3395875 stored elements in Compressed Sparse Row format>

In [15]:
# cross validation
from sklearn.model_selection import cross_val_score

model_mult = Mult.fit(X, y)

y_pred = model_mult.predict(X)
result = np.mean(y_pred == y)
result # 0.7309679984098588

score = cross_val_score(Mult, X, y, scoring='accuracy', cv=5)
score

array([0.64474774, 0.67167986, 0.67653879, 0.69315885, 0.68126553])

In [16]:
# change model
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score

Gaus = GaussianNB()
Bern = BernoulliNB()
Mult = MultinomialNB()

In [19]:
# modeling with cross validation

models = [Bern, Mult]

y = G_df.points.values
X
# X = X_sparse.todense()

for model in models:
    
    score = cross_val_score(model, X, y, scoring='accuracy', cv=5)
    
    print("{} model score is {}".format(model, score))

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True) model score is [0.69632623 0.72498095 0.73398264 0.75149909 0.74149412]
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True) model score is [0.64474774 0.67167986 0.67653879 0.69315885 0.68126553]


In [21]:
bern_result = np.mean([0.69632623, 0.72498095, 0.73398264, 0.75149909, 0.74149412])
mult_result = np.mean([0.64474774, 0.67167986, 0.67653879, 0.69315885, 0.68126553])

bern_result, mult_result

(0.729656606, 0.673478154)